In [4]:
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.utils.card_utils import gen_cards
from pypokerengine.players import BasePokerPlayer
from baseline_players import RandomPlayer
from bots.honest_player import HonestPlayer
from bots.honest_modified_player import HonestModifiedPlayer
from bots.RLbot import RLPokerPlayer
from bots.bot_denis_2 import BotDenisPlayer
from bots.bot1 import KoffaPlayer
from bots.forl_condition_bot_2 import ConditionPlayer
from bots.botDenis import GloomCha
from bots.forl_condition_bot import HonestStatPlayer
from bots.fish_player import FishPlayer
from bots.fold_player import FoldPlayer
from bots.bot_denis_3 import BotDenis3Player
from copy import deepcopy
import scipy.stats as sps
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [8]:
N_RANDOM_PLAYERS = 0

config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15)
# config.register_player(name="foldman", algorithm=FoldPlayer())
config.register_player(name="honest", algorithm=HonestPlayer(200))
config.register_player(name="honest_modified", algorithm=HonestModifiedPlayer(200))
config.register_player(name="fishman", algorithm=FishPlayer())
config.register_player(name="botDenis1", algorithm=GloomCha())
config.register_player(name="botDenis2", algorithm=BotDenisPlayer())
config.register_player(name="conditionPlayer", algorithm=ConditionPlayer())
config.register_player(name="koffa", algorithm=KoffaPlayer())
config.register_player(name="random", algorithm=RandomPlayer())

for i in range(N_RANDOM_PLAYERS):
    config.register_player(name="random" + str(i), algorithm=RandomPlayer())

In [10]:
%%time
from joblib import Parallel, delayed

def f(i,config):
    return start_poker(config, verbose=0)['players']

def get_all_results(config, n_games=1):
    players_stack_dict = {}
    for player_info in config.players_info:
        players_stack_dict[player_info['name']] = []
        
        
    with Parallel(n_jobs=4) as parallel:
#     for i in tqdm(range(n_games)):
        data = parallel(delayed(f)(i, config) for i in range(n_games))
    
    for i in tqdm(range(n_games)):
        for player_info in data[i]:
            players_stack_dict[player_info['name']].append(player_info['stack'])
            
    return pd.DataFrame(players_stack_dict)

res = get_all_results(config, n_games=30)






100%|██████████| 30/30 [00:00<00:00, 33788.70it/s]

CPU times: user 532 ms, sys: 174 ms, total: 706 ms
Wall time: 16min 33s


In [12]:
res.describe()

,botDenis1,botDenis2,conditionPlayer,fishman,foldman,honest,honest_modified,koffa,random
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.00000,30.000000
mean,2220.100000,3265.233333,1037.366667,1587.183333,1226.500000,879.976667,1285.400000,1510.60000,480.833333
std,2342.734967,3374.607182,1933.654562,3000.216717,111.395149,1758.441862,924.871794,352.56375,1359.559794
min,0.000000,0.000000,0.000000,0.000000,825.000000,0.000000,0.000000,1035.00000,0.000000
25%,1143.750000,0.000000,0.000000,0.000000,1185.000000,0.000000,213.750000,1165.87500,0.000000
50%,1342.500000,2507.500000,0.000000,0.000000,1230.000000,0.000000,1443.500000,1502.10000,0.000000
75%,2775.000000,5761.500000,714.000000,1648.125000,1286.250000,1032.375000,1771.250000,1781.25000,0.000000
max,9055.000000,10391.000000,5865.000000,11336.400000,1455.000000,6985.000000,3105.000000,2364.30000,5250.000000


In [ ]:
cnt = 0
class PrintPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"
    stats = PlayerStats()
    cnt = 0
    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        self.stats.update(hole_card, round_state)
        print(self.stats.calc_params(hole_card, round_state))
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        self.stats.init_player_names(game_info)
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [6]:
res.describe()

,fishman,foldman,honest,random0,random1,random2,random3,random4,random5
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,4158.633333,887.000000,2133.866667,24.433333,1032.233333,1666.566667,1490.666667,1260.933333,805.866667
std,5569.237944,106.338756,3427.695272,133.826878,3305.520614,4321.575652,3961.730529,3847.468240,2675.882474
min,0.000000,555.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,825.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,862.500000,1155.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,11102.750000,960.000000,1682.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,12902.000000,1110.000000,12523.000000,733.000000,12621.000000,12542.000000,12590.000000,12616.000000,12634.000000


In [ ]:
N = 100

RLBotPark = []
DefaultBotPark = [RandomPlayer, HonestPlayer, FishPlayer, 
           BotDenisPlayer, HonestModifiedPlayer, ConditionPlayer, 
           GloomCha, Coffa, HonestStatPlayer,
          ]

for i in tqdm(range(N)):
    in_model_file = None if i == 0 else 'model_{}.h5'.format(i)
    out_model_file = 'model_{}.h5'.format(i + 1)
    config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15)
    humans = np.random.choice(DefaultBotPark, replace=True, size=4 + min(0, len(RLBotPark) - 4))
    rlbots = np.random.choice(DefaultBotPark, replace=True, size=min(4, len(RLBotPark)))
    players = humans + rlbots + [None]
    players = np.random.shuffle(players)
    for player in players:
        config.register_player(name=palyer.__name__  if player is not None else 'RL-{}'.format(i), player())

AttributeError: 'NoneType' object has no attribute '__name__'